In [38]:
import pandas as pd
import pulp
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from mplsoccer import Pitch
import os
import numpy as np
from matplotlib.patches import FancyBboxPatch
from matplotlib import patheffects as path_effects

# 1. Read the data
player_stats = pd.read_csv('player_stats.csv')
players = pd.read_csv('players.csv')

# 2. Merge the data
df = pd.merge(players, player_stats, left_on='id', right_on='player_id', how='left')

# 3. Calculate total points (recency-weighted average)
gameweek_cols = [col for col in df.columns if col.startswith('gameweek_')]
# Sort by week index numerically (gameweek_1, gameweek_2, ...)

def _gw_key(c):
    try:
        return int(c.split('_')[-1])
    except Exception:
        return 0

gameweek_cols = sorted(gameweek_cols, key=_gw_key)

# Replace NaNs with 0 for safety
df[gameweek_cols] = df[gameweek_cols].fillna(0)

# Linear weights increasing towards most recent week
weights = np.arange(1, len(gameweek_cols) + 1, dtype=float)
weighted_sum = (df[gameweek_cols] * weights).sum(axis=1)
df['total_points'] = weighted_sum / weights.sum()

# 4. Create the optimization problem
prob = pulp.LpProblem("FantasyTeam", pulp.LpMaximize)

# 5. Decision variables
player_vars = pulp.LpVariable.dicts("Player", df.index, cat='Binary')
captain_var = pulp.LpVariable.dicts("Captain", df.index, cat='Binary')

# 6. Objective function
prob += pulp.lpSum([df.loc[i, 'total_points'] * (player_vars[i] + captain_var[i]) for i in df.index])

# 7. Position mapping
position_map = {
    'lib_gardien': 'GK',
    'lib_defenseur': 'DEF',
    'lib_milieu': 'MID',
    'lib_attaquant': 'FWD'
}
df['position_cat'] = df['position_code'].map(position_map)

# 8. Team composition
# We'll select 2 GKs, 5 DEFs, 4 MIDs, 4 FWDs in total
# Then pick lowest scorer from each position as substitute
total_positions = {
    'GK': 2,    # 1 starter, 1 sub
    'DEF': 5,   # 4 starters, 1 sub
    'MID': 4,   # 3 starters, 1 sub
    'FWD': 4    # 3 starters, 1 sub
}

starting_positions = {
    'GK': 1,
    'DEF': 4,
    'MID': 3,
    'FWD': 3
}

substitute_positions = {
    'GK': 1,
    'DEF': 1,
    'MID': 1,
    'FWD': 1
}

# 9. Club to kit mapping
club_to_kit = {
    'Celtic': 'm_sco_cec_2025.png',
    'Rangers': 'm_sco_ras_2025.png',
    'Hibernian': 'm_sco_hin_2025.png',
    'Heart of Midlothian': 'm_sco_min_2025.png',  
    'Dundee': 'm_sco_due_2025.png',
    'Aberdeen': 'm_sco_abn_2025_v2.png',
    'Dundee United': 'm_sco_dud_2025.png',
    'Falkirk': 'm_sco_fak_2025.png',
    'Kilmarnock': 'm_sco_kik_2025.png',
    'Livingston': 'm_sco_lin_2025_v2.png',
    'Motherwell': 'm_sco_mol_2025.png',
    'St. Mirren': 'm_sco_min_2025.png',
    'St. Johnstone': 'm_sco_stn_2025.png'
}

# 10. Constraints
# Budget constraint (100m)
prob += pulp.lpSum([df.loc[i, 'value'] * (player_vars[i] + captain_var[i]) for i in df.index]) <= 100

# Total players per position (starters + subs)
for pos, count in total_positions.items():
    prob += pulp.lpSum([player_vars[i] for i in df[df['position_cat'] == pos].index]) == count

# Max 4 players from the same club
for club in df['club'].unique():
    prob += pulp.lpSum([(player_vars[i] + captain_var[i]) for i in df[df['club'] == club].index]) <= 4

# Only one captain
prob += pulp.lpSum([captain_var[i] for i in df.index]) == 1

# A player can be either in the team or captain, not both
for i in df.index:
    prob += player_vars[i] + captain_var[i] <= 1

# 11. Solve the problem
prob.solve()


def add_kit(ax, club, x, y, zoom=0.25, is_captain=False):
    """Add kit image to the plot with the specified zoom level"""
    # Skip kit loading for placeholder clubs
    if club == 'N/A':
        color = 'gold' if is_captain else 'blue'
        player_circle = plt.Circle((x, y), 4, color=color, zorder=3)
        ax.add_patch(player_circle)
        return (0, 0, 3)
        
    try:
        kit_path = os.path.join('kits', club_to_kit[club])
        img = mpimg.imread(kit_path)
        im = OffsetImage(img, zoom=zoom, zorder=3)
        ab = AnnotationBbox(im, (x, y), frameon=False, pad=0, box_alignment=(0.5, 0.5))
        ax.add_artist(ab)
        return img.shape
    except Exception:
        # Fallback: draw a colored circle
        color = 'gold' if is_captain else 'blue'
        player_circle = plt.Circle((x, y), 4, color=color, zorder=3)
        ax.add_patch(player_circle)
        return (0, 0, 3)  # Return dummy shape


def create_football_field(starting_11, captain_id, substitutes=None,
                          total_points_ss_on=None, total_points_ss_off=None,
                          total_cost=None):
    # Single-pitch figure (no substitutes panel)
    fig = plt.figure(figsize=(20, 12))
    ax_pitch = fig.add_subplot(111)
    pitch = Pitch(pitch_type='statsbomb', line_color='black', pitch_color='#2e8b57')
    pitch.draw(ax=ax_pitch)
    
    # Starting 11 positions (x, y) in pitch coordinates
    # Symmetric 4-3-3 coordinates (StatsBomb: x 0-120, y 0-80)
    positions = {
        'GK': [(12, 40)],
        'DEF': [(28, 20), (28, 32), (28, 48), (28, 60)],  # LB, LCB, RCB, RB (symmetric around y=40)
        'MID': [(58, 25), (62, 40), (58, 55)],            # LCM, CM, RCM (balanced/centralized)
        'FWD': [(92, 25), (96, 40), (92, 55)]              # LW, ST, RW (symmetric and slightly advanced)
    }
    
    # Calculate totals (use provided adjusted totals if available)
    if (total_points_ss_on is None) or (total_points_ss_off is None) or (total_cost is None):
        # Fallback to simple sums if not provided
        total_points_ss_on = starting_11.get('total_points', pd.Series(dtype=float)).sum()
        total_points_ss_off = total_points_ss_on
        total_cost = starting_11.get('value', pd.Series(dtype=float)).sum()
        if substitutes is not None and not substitutes.empty:
            total_points_ss_on += substitutes.get('total_points', pd.Series(dtype=float)).sum()
            total_points_ss_off = total_points_ss_on
            total_cost += substitutes.get('value', pd.Series(dtype=float)).sum()
    
    # Plot starting 11 with proper spacing
    for pos in positions:
        players = starting_11[starting_11['position_cat'] == pos].head(len(positions[pos]))
        for (x, y), (_, player) in zip(positions[pos], players.iterrows()):
            is_captain = player['player_id'] == captain_id
            
            # Add kit with proper size
            _ = add_kit(ax_pitch, player['club'], x, y, zoom=0.22, is_captain=is_captain)
            
            # Player info below the kit (use offset points for consistent spacing)
            name = player['name'] + (" (C)" if is_captain else "")
            # Optional small captain badge just under the kit
            if is_captain:
                ax_pitch.plot(x, y-6, 'o', markersize=14, color='gold', zorder=4)
                ax_pitch.text(x, y-6, 'C', ha='center', va='center', fontsize=7,
                              fontweight='bold', color='black', zorder=5)

            line1 = f"{name}"
            # Display adjusted XI points if available; else raw total_points
            show_pts = player.get('adjusted_points', player.get('total_points', 0))
            line2 = f"{player['position_cat']} | {player['club']} | {show_pts:.2f} pts"
            info = f"{line1}\n{line2}"
            txt = ax_pitch.annotate(
                info,
                (x, y),
                xytext=(0, -26),
                textcoords='offset points',
                ha='center', va='top', fontsize=10, color='white',
                bbox=dict(facecolor='black', alpha=0.8,
                          edgecolor='white', boxstyle='round,pad=0.5'),
                zorder=6
            )
            # Extra stroke for readability
            txt.set_path_effects([
                path_effects.Stroke(linewidth=1.0, foreground='black'),
                path_effects.Normal()
            ])
    # No substitutes panel/images; only XI is drawn on pitch
    
    # Set title with total points and cost for the entire squad
    fig.suptitle('OPTIMAL FANTASY TEAM', fontsize=24, y=0.98)
    
    # Add formation and stats as subtitle
    ax_pitch.set_title(
        f"Formation: 4-3-3  |  Squad Total (SS plays): {total_points_ss_on:.2f} pts  |  (SS doesn't play): {total_points_ss_off:.2f} pts  |  Cost: {total_cost:.2f}m",
        fontsize=12, pad=20
    )
    
    # Adjust layout and save
    plt.tight_layout(rect=[0, 0, 1, 0.96])  # Make room for suptitle
    plt.savefig('team_formation.png', dpi=300, bbox_inches='tight', facecolor='white')
    plt.close()

# Get selected players
selected_team = []
captain_id = None
starting_players = []

# Get all selected players
selected_indices = [i for i in df.index if player_vars[i].value() == 1]
all_players = [df.loc[i].copy() for i in selected_indices]

# Sort players by position and then by points (descending)
all_players.sort(key=lambda x: (x['position_cat'], -x['total_points']))

# Group players by position
position_players = {pos: [] for pos in ['GK', 'DEF', 'MID', 'FWD']}
for player in all_players:
    position_players[player['position_cat']].append(player)

# Select starting 11 (top scorers from each position)
starting_players = []
for pos, count in starting_positions.items():
    starting_players.extend(position_players[pos][:count])  # Take top N for starting positions

# Select substitutes (lowest scorers from each position not in XI)
substitute_players = []
for pos in ['GK', 'DEF', 'MID', 'FWD']:
    all_in_pos = position_players[pos]
    starting_in_pos = [p for p in starting_players if p['position_cat'] == pos]
    starting_players_set = {tuple(sorted(p.items())) for p in starting_in_pos}
    available_subs = [p for p in all_in_pos if tuple(sorted(p.items())) not in starting_players_set]
    if available_subs:
        lowest_scoring = min(available_subs, key=lambda x: x['total_points'])
        substitute_players.append(lowest_scoring)
    else:
        substitute_players.append({
            'name': f'No {pos} sub',
            'position_cat': pos,
            'club': 'N/A',
            'total_points': 0,
            'value': 0,
            'player_id': f'no_{pos.lower()}_sub'
        })

# Select captain - highest scoring player in starting 11
if starting_players:
    captain = max(starting_players, key=lambda x: x['total_points'])
    captain_id = captain['player_id']
    # Double points for captain (we'll recover base later)
    captain['total_points'] *= 2

# Create DataFrames for XI and subs
starting_11 = pd.DataFrame(starting_players)
subs = pd.DataFrame(substitute_players)
if not subs.empty and 'position_cat' in subs.columns:
    subs = subs.sort_values('position_cat')

###############################################
# Supersub logic and adjusted points
###############################################

# Mark captain and compute base/adjusted XI points
if not starting_11.empty:
    starting_11['is_captain'] = starting_11['player_id'] == captain_id
    # Because we previously doubled the captain's total_points, recover base_points for XI
    starting_11['base_points'] = np.where(starting_11['is_captain'],
                                          starting_11['total_points'] / 2.0,
                                          starting_11['total_points'])
    starting_11['adjusted_points'] = np.where(starting_11['is_captain'],
                                              starting_11['base_points'] * 2.0,
                                              starting_11['base_points'])
else:
    starting_11['is_captain'] = False
    starting_11['base_points'] = 0.0
    starting_11['adjusted_points'] = 0.0

# Prepare subs base points
if not subs.empty:
    subs['is_supersub'] = False
    subs['base_points'] = subs['total_points']
    # Choose Supersub: highest projected points among subs
    choose_supersub_highest = True
    if choose_supersub_highest and not subs['base_points'].empty:
        ss_idx = subs['base_points'].astype(float).idxmax()
        subs.loc[ss_idx, 'is_supersub'] = True
    # Apply multipliers for two scenarios
    subs['adjusted_points_ss_off'] = subs['base_points'] * 0.5
    subs['adjusted_points_ss_on'] = np.where(subs['is_supersub'], subs['base_points'] * 3.0, subs['base_points'] * 0.5)
else:
    subs['is_supersub'] = False
    subs['base_points'] = 0.0
    subs['adjusted_points_ss_off'] = 0.0
    subs['adjusted_points_ss_on'] = 0.0

# Totals for both scenarios
xi_total = starting_11['adjusted_points'].sum() if 'adjusted_points' in starting_11 else 0.0
subs_total_off = subs['adjusted_points_ss_off'].sum() if 'adjusted_points_ss_off' in subs else 0.0
subs_total_on = subs['adjusted_points_ss_on'].sum() if 'adjusted_points_ss_on' in subs else 0.0
total_points_ss_off = xi_total + subs_total_off
total_points_ss_on = xi_total + subs_total_on

# Total cost
total_cost = starting_11.get('value', pd.Series(dtype=float)).sum()
if not subs.empty:
    total_cost += subs.get('value', pd.Series(dtype=float)).sum()

# Consolidated team DF with roles and adjusted points for both scenarios
starting_11_out = starting_11.copy()
starting_11_out['role'] = np.where(starting_11_out['is_captain'], 'XI (C)', 'XI')
starting_11_out['adjusted_points_ss_off'] = starting_11_out['adjusted_points']
starting_11_out['adjusted_points_ss_on'] = starting_11_out['adjusted_points']

subs_out = subs.copy()
subs_out['role'] = np.where(subs_out['is_supersub'], 'SUB (SS)', 'SUB')

team_df = pd.concat([starting_11_out, subs_out], ignore_index=True)

required_positions = ['GK', 'DEF', 'MID', 'FWD']

# Create and save the football field with dual totals (Supersub on/off)
create_football_field(starting_11_out, captain_id, subs_out,
                      total_points_ss_on=total_points_ss_on,
                      total_points_ss_off=total_points_ss_off,
                      total_cost=total_cost)

# Print team summary
print("\n🎯 OPTIMAL TEAM SELECTION 🎯")
print("="*50)
print(f"Total Cost: {total_cost:.2f}m")
print(f"Total Points (SS plays): {total_points_ss_on:.2f}")
print(f"Total Points (SS doesn't play): {total_points_ss_off:.2f}")

# Print starting 11
print("\n⚽ STARTING 11 ⚽")
print("-"*50)
for pos in ['GK', 'DEF', 'MID', 'FWD']:
    for _, player in starting_11_out[starting_11_out['position_cat'] == pos].iterrows():
        captain = "(C) " if player['player_id'] == captain_id else ""
        pts = player.get('adjusted_points', player.get('total_points', 0))
        print(f"{player['position_cat']}: {captain}{player['name']} ({player['club']}) - {pts:.2f}pts")

print("\n🔹 SUBSTITUTES 🔹")
print("-"*50)
for i, (_, sub) in enumerate(subs_out.iterrows(), 1):
    tag = " [SS]" if sub.get('is_supersub', False) else ""
    club = sub.get('club', 'N/A')
    off_pts = sub.get('adjusted_points_ss_off', sub.get('total_points', 0) * 0.5)
    on_pts = sub.get('adjusted_points_ss_on', sub.get('total_points', 0) * (3.0 if sub.get('is_supersub', False) else 0.5))
    print(f"{i}. {sub['name']}{tag} ({sub['position_cat']}, {club}) - off: {off_pts:.2f} | on: {on_pts:.2f} pts")

print("\n✅ Team formation saved as 'team_formation.png'")
print("📊 Team details saved to 'optimal_team.csv'")

# Save the team to CSV (rounded to 2 decimals for numeric columns)
team_df_rounded = team_df.copy()
for col in team_df_rounded.select_dtypes(include=[np.number]).columns:
    team_df_rounded[col] = team_df_rounded[col].round(2)
team_df_rounded.to_csv('optimal_team.csv', index=False)



🎯 OPTIMAL TEAM SELECTION 🎯
Total Cost: 90.60m
Total Points (SS plays): 145.83
Total Points (SS doesn't play): 129.58

⚽ STARTING 11 ⚽
--------------------------------------------------
GK: K. Schmeichel (Celtic) - 6.00pts
DEF: S. Findlay (Heart of Midlothian) - 11.33pts
DEF: H. Milne (Heart of Midlothian) - 10.17pts
DEF: R. Astley (Dundee) - 7.50pts
DEF: B. Brannan (Kilmarnock) - 7.50pts
MID: E. Just (Motherwell) - 7.83pts
MID: T. Maswanhise (Motherwell) - 7.33pts
MID: S. Pittman (Livingston) - 6.83pts
FWD: (C) B. Nygren (Celtic) - 33.67pts
FWD: C. Braga (Heart of Midlothian) - 10.17pts
FWD: I. Dolcek (Dundee United) - 9.17pts

🔹 SUBSTITUTES 🔹
--------------------------------------------------
1. J. Tavernier [SS] (DEF, Rangers) - off: 3.25 | on: 19.50 pts
2. J. Ayunga (FWD, St. Mirren) - off: 2.75 | on: 2.75 pts
3. J. McCracken (GK, Dundee) - off: 2.83 | on: 2.83 pts
4. F. Curtis (MID, Rangers) - off: 3.25 | on: 3.25 pts

✅ Team formation saved as 'team_formation.png'
📊 Team details 